## Importing performance data, initial cleaning, converting data into dictionary (combined_data_), datafram with dropped kernel name for analysis (df2), and normalized data (X) to be used in later analysis

In [1]:
# Load libraries
import numpy as np
from keras.layers import Input,Dense
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import pandas as pd
import sys
import os
import glob
import csv
import os.path

# Step 1 - Process Data:

save_fname = "saved_data"
if os.path.isfile(save_fname+".npy"):
    print("Loading saved X data.")
    X = np.load(save_fname+".npy")

else:
    #calculating number of kernels total
    kernel_count = 0
    total_kernels = []
    combined_data_ = {}
    target_kernels = []
    metric_targets = []
    all_sig_metrics = []
    for filen_ in glob.glob("/Users/yzamora/Desktop/all_data/*.csv"):

        filen = os.path.basename(filen_)
        filen_split = filen.split('.')[0].split('_')
        bench_name = filen_split[1]
        architecture = filen_split[0]
        unique_spec = filen_split[2]
        
        key_root = bench_name
        levels = ["Idle", "Low","High", "Max"]
        bw_units = ["GB", "MB", "KB" ,"0B"]

        # Now open the file and look for the data
        with open(filen_ ,'r') as file_handle:

            data_found = False
            ncols = 1
            fdata = csv.reader(file_handle)
            index_lookup = {}
            for line_split in fdata:

                lsplt = (len(line_split) > 0)

                if data_found:

                    if lsplt and len(line_split) == ncols:

                        # Read in desired value for the current metric
                        target_index = index_lookup['Avg']; value = 0
                        metric_name = line_split[index_lookup['Metric Name']]
                        if line_split[target_index].isdecimal():
                            if line_split[target_index]!= '0':

                                all_sig_metrics.append(metric_name)
                                value = int(line_split[ target_index ])

                            # Labeled with percentage
                        elif "%" == line_split[target_index][-1]:
                            #print ("percentage loop")
                            all_sig_metrics.append(metric_name)
                            value = float(line_split[ target_index ][0:7]) / 100.0

                        # Labeled with bandwidth units
                        elif line_split[ target_index ][-4:-2] in bw_units:
                            # Just take the first
                            units = line_split[ target_index ][-4:-2]
                            all_sig_metrics.append(metric_name)
                            mfact = 1.0
                            if   units == "KB": mfact = 1024
                            elif units == "MB": mfact = 1024*1024
                            elif units == "GB": mfact = 1024*1024*1024
                            elif units == "0B":  mfact = 1
                            value = float(line_split[ target_index ][0:7]) * mfact

                        # idle, low, max
                        elif line_split[ target_index ][-1] == ")":
                            #print ("low")
                            all_sig_metrics.append(metric_name)
                            value = int(line_split[ target_index].split('(')[1].split(")")[0])

                        # otherwise, float
                        else:
                            value = float(line_split[ target_index ])

                         # Parse name of kernel
                        kernel_name = line_split[ index_lookup['Kernel'] ].split('(')[0]
                        if not(kernel_name in total_kernels):
                            total_kernels.append(kernel_name)
                            kernel_count += 1

                        # Define kernel-specific key
                        key = key_root + architecture + "_" + unique_spec + "_" + kernel_name

                        # Initialize dict for this key, if it is new
                        if not (key in combined_data_):
                            combined_data_ [ key ] = {}
                        if not (kernel_name in target_kernels):
                            target_kernels.append(kernel_name)

                        # Store value for the metric being read right now
                        combined_data_[key][ metric_name ] = value
                        combined_data_[key]["kernelname"] = kernel_name

                    else:
                        data_found = False


                elif lsplt and line_split[0] == 'Device' and line_split[1] == 'Kernel':
                    # Set flag that we are at the data:
                    data_found = True
                    # Set number of columns in table:
                    ncols = len(line_split)
                    # Generate an index lookup table:
                    idx = 0
                    for term in line_split:
                        index_lookup[term] = idx
                        idx += 1
    #from sklearn.preprocessing import scale
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import MinMaxScaler

    df = pd.DataFrame.from_dict(combined_data_,orient='index')
    df = df.dropna(axis=1,how='any')
    df2 = df.drop(columns=['kernelname'])
    data = df2.values #scale(df2.values)
    #X = StandardScaler().fit_transform(data)
    X = MinMaxScaler().fit_transform(data)
    #X=X.astype('float32')/float(X.max())

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Data information 

In [6]:
print("There are %i data points." % len(X))
print("There are %i features." % len(X[0]))
print("There are %i kernels tested." % kernel_count)

There are 13866 data points.
There are 118 features.
There are 29 kernels tested.
